# Proyecto 1 - Tópicos Avanzados en Inteligencia Artificial - Grupo 3

## Desarrollado por: Demier de Jesús Ballesteros Padila y Simón Dávila Saravia


# 2. Descripción y carga del Dataset

El conjunto de datos se utiliza para predecir el tipo de cobertura forestal a partir de variables cartográficas. El conjunto de datos contiene información de 4 áreas silvestres ubicadas en el bosque nacional Roosevelt al norte de Colorado (Estados Unidos). Las áreas estudiadas son tipos de cobertura forestal resultado del proceso ecológcio.

Para el proyecto en cuestión, se utilizará una versión modificada del conjunto de datos.

## 2.1 Cargue del Dataset

In [1]:
import os
import requests

# Ubicación del almacenamiento de metada del pipline
_pipeline_root = './pipeline/'

# Directorio del conjunto de datos sin procesar
_data_root = './data/covertype'

# Ruta al conjunto de datos de entrenamiento
_data_filepath = os.path.join(_data_root, 'covertype.train.csv')

# Descarga del conjunto de datos
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

# 3. Selección de características

In [ ]:
import pandas as pd
from sklearn.feature_selection import GenericUnivariateSelect, SelectKBest,f_classif
from sklearn.model_selection import train_test_split

# Se instancia el data frame con el conjunto de datos
df = pd.read_csv('data/covertype/covertype.train.csv', index_col=False)
df.head()

In [3]:
# Eliminación de las características categóricas
df_ct = df.drop(['Wilderness_Area', 'Soil_Type'], axis=1)
df_ct.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,1
1,2876,3,18,485,71,2495,192,202,144,1557,1
2,3171,315,2,277,9,4374,213,237,162,1052,0
3,3087,342,13,190,31,4774,193,221,166,752,0
4,2835,158,10,212,41,3596,231,242,141,3280,1


In [4]:
# Selección de las mejores características
y = df_ct['Cover_Type']
X = df_ct.drop(['Cover_Type'], axis=1)

select_k_best = SelectKBest(f_classif, k=8)
X_new = select_k_best.fit_transform(X,y)

features_mask = select_k_best.get_support()

In [5]:
# Resultado de la selecciónd de las mejores 8 características

print("{:<40} {:<10}".format('Columns', 'Retain'))
for bool_val, feature in zip(features_mask, X.columns):
        print("{:<40} {}".format(feature, bool_val))

Columns                                  Retain    
Elevation                                True
Aspect                                   False
Slope                                    True
Horizontal_Distance_To_Hydrology         True
Vertical_Distance_To_Hydrology           True
Horizontal_Distance_To_Roadways          True
Hillshade_9am                            True
Hillshade_Noon                           True
Hillshade_3pm                            False
Horizontal_Distance_To_Fire_Points       True


In [6]:
# pd.DataFrame(features_mask).to_csv("data/covertype/prueba.csv")
features_names = select_k_best.get_feature_names_out()
print(features_names, X_new)

['Elevation' 'Slope' 'Horizontal_Distance_To_Hydrology'
 'Vertical_Distance_To_Hydrology' 'Horizontal_Distance_To_Roadways'
 'Hillshade_9am' 'Hillshade_Noon' 'Horizontal_Distance_To_Fire_Points'] [[2991    7   67 ...  233  234 1570]
 [2876   18  485 ...  192  202 1557]
 [3171    2  277 ...  213  237 1052]
 ...
 [3166   11   67 ...  234  240 2404]
 [3154   14  738 ...  181  239 1320]
 [3031    4  426 ...  227  237 1388]]


In [7]:
features_names.shape
features_names.tolist()
df_ct = pd.DataFrame(X_new, columns = features_names)
df_ct.insert(len(df_ct.columns),'Cover_Type', y)

In [8]:

_process_data_root = './data/covertype/process'
if not os.path.exists(_process_data_root):
    os.makedirs(_process_data_root)
pd.DataFrame(df_ct).to_csv("data/covertype/process/covertype.csv", index=False)

# 4. Data Pipeline

In [9]:
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.v1.components import ImportSchemaGen
from tfx.dsl.components.common.importer import Importer
import tensorflow_data_validation as tfdv
import tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2

import pprint
pp = pprint.PrettyPrinter()

2023-03-08 23:16:05.036892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 23:16:05.222388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 23:16:05.222414: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-08 23:16:06.392812: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## 4.1 Configurar el context interactivo

In [10]:
# Inicialización del contexto interactivo
contexto = InteractiveContext(pipeline_root=_pipeline_root)

## 4.2 Generando Ejemplos

In [11]:
# Se instancia el generador de ejemplos, usando la ruta del conjunto de datos como entrada
gen_ejemplo = CsvExampleGen(input_base=_process_data_root)
contexto.run(gen_ejemplo)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [12]:
#Obtener el artefacto
artefacto = gen_ejemplo.outputs['examples'].get()[0]

#Imprimir split names y uri
print(f' split names: {artefacto.split_names}')
print(f' uri del artefacto: {artefacto.uri}')

 split names: ["train", "eval"]
 uri del artefacto: ./pipeline/CsvExampleGen/examples/1


In [13]:
uri_entrenamiento = os.path.join(artefacto.uri, 'Split-train')

!ls {uri_entrenamiento}

data_tfrecord-00000-of-00001.gz


In [14]:
tfrecord_nombresarchivos = [os.path.join(uri_entrenamiento, name)
                            for name in os.listdir(uri_entrenamiento)]

conjuntodatos = tf.data.TFRecordDataset(tfrecord_nombresarchivos, compression_type="GZIP")

2023-03-08 23:16:50.710801: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-08 23:16:50.710842: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-08 23:16:50.710874: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (25b63f8c4ef4): /proc/driver/nvidia/version does not exist
2023-03-08 23:16:50.711646: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
def get_registros(conjuntodatos, num_registros):
    
    registros = []
    
    for tfrecord in conjuntodatos.take(num_registros):
        ejemplo_serealizado = tfrecord.numpy()
        
        ejemplo = tf.train.Example()
        
        ejemplo.ParseFromString(ejemplo_serealizado)
        
        diccionario_ejemplo = (MessageToDict(ejemplo))
        
        registros.append(diccionario_ejemplo)
        
    return registros

In [16]:
muestra_registros = get_registros(conjuntodatos, 3)

pp.pprint(muestra_registros)

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2991']}},
                           'Hillshade_9am': {'int64List': {'value': ['233']}},
                           'Hillshade_Noon': {'int64List': {'value': ['234']}},
                           'Horizontal_Distance_To_Fire_Points': {'int64List': {'value': ['1570']}},
                           'Horizontal_Distance_To_Hydrology': {'int64List': {'value': ['67']}},
                           'Horizontal_Distance_To_Roadways': {'int64List': {'value': ['1015']}},
                           'Slope': {'int64List': {'value': ['7']}},
                           'Vertical_Distance_To_Hydrology': {'int64List': {'value': ['11']}}}}},
 {'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2876']}},
                           'Hillshade_9am': {'int64List': {'value': ['192']}},

## 4.3 Estadísticas

In [17]:
gen_estadisticas = StatisticsGen(examples = gen_ejemplo.outputs['examples'])

contexto.run(gen_estadisticas)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [18]:
contexto.show(gen_estadisticas.outputs['statistics'])

## 4.4 Inferir el esquema

In [19]:
gen_esquema = SchemaGen( statistics = gen_estadisticas.outputs['statistics'])
contexto.run(gen_esquema)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [20]:
contexto.show(gen_esquema.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-


In [21]:
artefacto_esquema = gen_esquema.outputs['schema'].get()[0]

In [22]:
print(artefacto_esquema.uri)

./pipeline/SchemaGen/schema/3


## 4.5 Curando el esquema

In [26]:
esquema_proto_path = '{}/{}'.format(artefacto_esquema.uri, 'schema.pbtxt')
esquema = tfdv.load_schema_text(esquema_proto_path)
tfdv.set_domain(esquema,'Hillshade_9am', schema_pb2.IntDomain(name='Hillshade_9am', min=0, max=255))
tfdv.set_domain(esquema,'Hillshade_Noon', schema_pb2.IntDomain(name='Hillshade_Noon', min=0, max=255))
tfdv.set_domain(esquema,'Slope', schema_pb2.IntDomain(name='Slope', min=0, max=90))
tfdv.set_domain(esquema,'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))

tfdv.display_schema(schema=esquema)
                


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


In [27]:
# tf.io.gfile.makedirs(esquema_dir)
archivo_esquema = os.path.join(artefacto_esquema.uri, 'schema.pbtxt')
tfdv.write_schema_text(esquema, archivo_esquema)


In [28]:
imp_esquema = Importer(
    source_uri=artefacto_esquema.uri,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False)

contexto.run(imp_esquema)

ExecutionResult(
    component_id: Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema:2023-03-08T23:17:22.103944"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.12.0"
          }
        }
        state: LIVE
        name: "schema:2023-03-08T23:17:22.103944"
        create_time_since_epoch: 1678335442140
        last_update_time_since_epoch: 1678335442157
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [29]:
contexto.show(imp_esquema.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


## 4.6 Entornos de esquema

In [30]:
if not os.path.exists(_process_data_root):
    os.makedirs(_process_data_root)
pd.DataFrame(X_new).to_csv("data/covertype/process/covertype_train.csv", header = features_names, index=False)



In [31]:
datos_servicio = os.path.join(_process_data_root,'covertype_train.csv')
estadisticas_servicio = tfdv.generate_statistics_from_csv(datos_servicio)
anomalias_servicio = tfdv.validate_statistics(estadisticas_servicio, esquema)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [32]:
tfdv.display_anomalies(anomalias_servicio)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


In [33]:
esquema.default_environment.append('ENTRENAMIENTO')
esquema.default_environment.append('SERVICIO')

tfdv.get_feature(esquema,'Cover_Type').not_in_environment.append('SERVICIO')

anomalias_servicio_con_entorno = tfdv.validate_statistics (estadisticas_servicio, esquema, environment='SERVICIO')
tfdv.display_anomalies(anomalias_servicio_con_entorno)



In [34]:
tfdv.display_schema(schema=esquema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


In [35]:
tfdv.write_schema_text(esquema, archivo_esquema)

In [36]:
imp_esquema_ent = Importer(
    source_uri=artefacto_esquema.uri,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False)

contexto.run(imp_esquema_ent)

ExecutionResult(
    component_id: Importer
    execution_id: 5
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema:2023-03-08T23:17:22.103944"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.12.0"
          }
        }
        state: LIVE
        name: "schema:2023-03-08T23:17:22.103944"
        create_time_since_epoch: 1678335442140
        last_update_time_since_epoch: 1678335442157
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [37]:
contexto.show(imp_esquema_ent.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


In [38]:
!cat {archivo_esquema}

feature {
  name: "Cover_Type"
  type: INT
  int_domain {
    name: "Cover_Type"
    min: 0
    max: 6
    is_categorical: true
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  not_in_environment: "SERVICIO"
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Elevation"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_9am"
  type: INT
  int_domain {
    name: "Hillshade_9am"
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_Noon"
  type: INT
  int_domain {
    name: "Hillshade_Noon"
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Horizontal_Distance_To_Fire_Points"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size

## 4.7 Genere nuevas estadísticas usando el esquema actualizado

In [39]:
esquema_gen_act = ImportSchemaGen(schema_file=archivo_esquema)

In [40]:
contexto.run(esquema_gen_act)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 6
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [41]:
contexto.show(esquema_gen_act.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


In [42]:
estadisticas_gen_act = StatisticsGen(examples = gen_ejemplo.outputs['examples'],schema=esquema_gen_act.outputs['schema'])

contexto.run(estadisticas_gen_act)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 7
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [43]:
contexto.show(estadisticas_gen_act.outputs['statistics'])

## 4.8 Comprobar anomalías

In [44]:
validador_ejemplos = ExampleValidator(
    statistics=estadisticas_gen_act.outputs['statistics'],
    schema=esquema_gen_act.outputs['schema'])

contexto.run(validador_ejemplos)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 8
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [45]:
contexto.show(validador_ejemplos.outputs['anomalies'])

## 4.9 Ingeniería de características

In [46]:
_constantes_covert_type = 'cover_type_constants.py'

In [47]:
%%writefile {_constantes_covert_type}

LLAVE_CARACTERISTICAS_NUMERICAS = ['Elevation', 'Horizontal_Distance_To_Fire_Points', 'Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Vertical_Distance_To_Hydrology']

LLAVE_RANGO_0_255 = ['Hillshade_9am', 'Hillshade_Noon']

LLAVE_RANGO_0_90 = ['Slope']

LABEL_KEY = 'Cover_Type'

def nombre_transformado(llave):
    return llave + '_xf'

Writing cover_type_constants.py


## 4.10 Función de preprocesamiento 

In [48]:
_transformar_cover_type = 'cover_type_transform.py'

In [49]:
%%writefile {_transformar_cover_type}

import tensorflow as tf
import tensorflow_transform as tft

import cover_type_constants

_LLAVE_CARACTERISTICAS_NUMERICAS = cover_type_constants.LLAVE_CARACTERISTICAS_NUMERICAS
_LLAVE_RANGO_0_255 = cover_type_constants.LLAVE_RANGO_0_255
_LLAVE_RANGO_0_90 = cover_type_constants.LLAVE_RANGO_0_90
_LABEL_KEY = cover_type_constants.LABEL_KEY
_nombre_transofrmado = cover_type_constants.nombre_transformado


def preprocessing_fn(entradas):
    
    salidas = {}
    
    for llave in _LLAVE_CARACTERISTICAS_NUMERICAS:
        salidas[_nombre_transofrmado(llave)] = tft.scale_to_0_1(
            entradas[llave])
    
    for llave in _LLAVE_RANGO_0_255:
        salidas[_nombre_transofrmado(llave)] = tft.scale_by_min_max(
            entradas[llave], 0.0, 255.0)
        
    for llave in _LLAVE_RANGO_0_90:
        salidas[_nombre_transofrmado(llave)] = tft.scale_by_min_max(
            entradas[llave], 0.0, 90.0)
    
    salidas[_nombre_transofrmado(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(entradas[_LABEL_KEY])
    
    
    return salidas

Writing cover_type_transform.py


## 4.11 Transformar

In [50]:
tf.get_logger().setLevel('ERROR')

transformacion = Transform(
    examples=gen_ejemplo.outputs['examples'],
    schema=esquema_gen_act.outputs['schema'],
    module_file=os.path.abspath(_transformar_cover_type))

contexto.run(transformacion)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying cover_type_constants.py -> build/lib
copying cover_type_transform.py -> build/lib
installing to /tmp/tmpzxivn6ri
running install
running install_lib
copying build/lib/cover_type_constants.py -> /tmp/tmpzxivn6ri
copying build/lib/cover_type_transform.py -> /tmp/tmpzxivn6ri
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpzxivn6ri/tfx_user_code_Transform-0.0+2df7714e8553cb2318e6a943693e51726f3512b627646950

Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+2df7714e8553cb2318e6a943693e51726f3512b6276469503e62cf9479513c00-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+2df7714e8553cb2318e6a943693e51726f3512b6276469503e62cf9479513c00-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 9
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [52]:
grafico_transformacion_uri = transformacion.outputs['transform_graph'].get()[0].uri

os.listdir(grafico_transformacion_uri)

['transformed_metadata', 'metadata', 'transform_fn']

In [53]:
uri_entrenamiento = os.path.join(transformacion.outputs['transformed_examples'].get()[0].uri, 'Split-train')

archivos_tfrecord = [os.path.join(uri_entrenamiento, nombre)
                     for nombre in os.listdir(uri_entrenamiento)]

conjunto_transformado = tf.data.TFRecordDataset(archivos_tfrecord, compression_type="GZIP")

In [54]:
def obtener_registros(conjunto, num_registros):
    
    registros = []
    
    for tfrecord in conjunto.take(num_registros):
        
        ejemplo_serializado = tfrecord.numpy()
        
        ejemplo = tf.train.Example()
        
        ejemplo.ParseFromString(ejemplo_serializado)
        
        diccionario_ejemplo = (MessageToDict(ejemplo))
        
        registros.append(diccionario_ejemplo)
        
    return registros

In [55]:
registros_muestra_xf = obtener_registros(conjunto_transformado, 3)

pp.pprint(registros_muestra_xf)

[{'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation_xf': {'floatList': {'value': [0.5686275]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [233.91731]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [234.92125]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.21902902]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.04886944]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [0.14263631]}},
                           'Slope_xf': {'floatList': {'value': [9.545455]}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.2214854]}}}}},
 {'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation_xf': {'floatList': {'value': [0.5108095]}},
       

# 5 Metadatos de aprendizaje automático

In [57]:
_util = 'util.py'

In [58]:
%%writefile {_util}

import tensorflow as tf
import pandas as pd
from google.protobuf.json_format import MessageToDict

def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

def display_types(types):
    # Helper function to render dataframes for the artifact and execution types
    table = {'id': [], 'name': []}
    for a_type in types:
        table['id'].append(a_type.id)
        table['name'].append(a_type.name)
    return pd.DataFrame(data=table)

def display_artifacts(store, artifacts, base_dir):
    # Helper function to render dataframes for the input artifacts
    table = {'artifact id': [], 'type': [], 'uri': []}
    for a in artifacts:
        table['artifact id'].append(a.id)
        artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
        table['type'].append(artifact_type.name)
        table['uri'].append(a.uri.replace(base_dir, './'))
    return pd.DataFrame(data=table)

def display_properties(store, node):
    # Helper function to render dataframes for artifact and execution properties
    table = {'property': [], 'value': []}
    
    for k, v in node.properties.items():
        table['property'].append(k)
        table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
    
    for k, v in node.custom_properties.items():
        table['property'].append(k)
        table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
    
    return pd.DataFrame(data=table)

Writing util.py


In [85]:
import util as u_t
import ml_metadata as mlmd
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'pipeline/metadata.sqlite'
connection_config.sqlite.connection_mode = 3
store = metadata_store.MetadataStore(connection_config)

## 5.1 Acceso a artefactos almacenados

In [61]:
artifact_types = store.get_artifact_types()

### Mostrando tipos de artefactos

In [64]:
u_t.display_types(artifact_types)

,id,name
0,14,Examples
1,16,ExampleStatistics
2,18,Schema
3,22,ExampleAnomalies
4,24,TransformGraph
5,25,TransformCache


### Lista de artefactos para un tipo particular (Schema, Examples, TransformGraph, ExampleStatistics)

In [66]:
list_artifact_type_schema = store.get_artifacts_by_type('Schema')
list_artifact_type_example = store.get_artifacts_by_type('Examples')
list_artifact_type_transform = store.get_artifacts_by_type('TransformGraph')
list_artifact_type_example_statistic = store.get_artifacts_by_type('ExampleStatistics')

### Mostrando la lista con la funcion de utilidad para visualizarlo

In [68]:
u_t.display_artifacts(store, list_artifact_type_schema, '.')

,artifact id,type,uri
0,3,Schema,.//pipeline/SchemaGen/schema/3
1,4,Schema,.//pipeline/ImportSchemaGen/schema/6
2,10,Schema,.//pipeline/Transform/pre_transform_schema/9
3,12,Schema,.//pipeline/Transform/post_transform_schema/9


In [69]:
u_t.display_artifacts(store, list_artifact_type_example, '.')

,artifact id,type,uri
0,1,Examples,.//pipeline/CsvExampleGen/examples/1
1,8,Examples,.//pipeline/Transform/transformed_examples/9


In [70]:
u_t.display_artifacts(store, list_artifact_type_transform, '.')

,artifact id,type,uri
0,7,TransformGraph,.//pipeline/Transform/transform_graph/9


In [71]:
u_t.display_artifacts(store, list_artifact_type_example_statistic, '.')

,artifact id,type,uri
0,2,ExampleStatistics,.//pipeline/StatisticsGen/statistics/2
1,5,ExampleStatistics,.//pipeline/StatisticsGen/statistics/7
2,11,ExampleStatistics,.//pipeline/Transform/pre_transform_stats/9
3,13,ExampleStatistics,.//pipeline/Transform/post_transform_stats/9


### Mostrando propiedades de un artefacto en particular para un nodo especifico. En este caso artifact_id=3

In [73]:
u_t.display_properties(store, list_artifact_type_schema[0])

,property,value
0,name,schema:2023-03-08T23:17:22.103944
1,state,published
2,producer_component,SchemaGen
3,tfx_version,1.12.0


### Mostrando propiedades de un artefacto en particular para un nodo especifico. En este caso artifact_id=7

In [75]:
u_t.display_properties(store, list_artifact_type_transform[0])

,property,value
0,name,transform_graph:2023-03-08T23:20:09.065686
1,tfx_version,1.12.0
2,producer_component,Transform
3,state,published


In [76]:
u_t.display_properties(store, list_artifact_type_example_statistic[0])

,property,value
0,split_names,"[""train"", ""eval""]"
1,tfx_version,1.12.0
2,state,published
3,name,statistics:2023-03-08T23:17:05.299234
4,producer_component,StatisticsGen


## 5.2 Seguimiento de artefactos

### Se crea función para obtener artefactos padres a partir de una artefacto dado

In [86]:
def get_main_artifacts(store, artifac_id):
    
    artifact_ids_list = list(map(int, str(artifac_id)))
    events = store.get_events_by_artifact_ids(artifact_ids_list)
    execution_ids = set()
    for event in events:
        if(event.type == metadata_store_pb2.Event.OUTPUT):
            execution_ids.add(event.execution_id)
    
    event_execution_ids = store.get_events_by_execution_ids(execution_ids)
    artifact_ids = set()
    for event_exc_id in event_execution_ids:
        if(event_exc_id.type == metadata_store_pb2.Event.INPUT):
            artifact_ids.add(event_exc_id.artifact_id)
            
    parent_artifacts = [art for art in store.get_artifacts_by_id(artifact_ids)]
    
    return parent_artifacts




## 5.3 Obtener artefactos principales

### Ejecutando funcion para traer artefactos principales del artefacto TransformGraph

In [88]:
artifact_transform = store.get_artifacts_by_type('TransformGraph')
_id = artifact_transform[0].id
artifacts_p = get_main_artifacts(store, _id)
u_t.display_artifacts(store, artifacts_p, '.')


,artifact id,type,uri
0,1,Examples,.//pipeline/CsvExampleGen/examples/1
1,4,Schema,.//pipeline/ImportSchemaGen/schema/6
